# TPC4:  Ficheiros CSV com listas e funções de agregação

Cria um programa em Python que implementa um conversor de um ficheiro CSV (Comma separated values) para o formato JSON.
Para se poder realizar a conversão pretendida, é importante saber que a primeira linha do CSV dado funciona como cabeçalho
que define o que representa cada coluna.
Por exemplo, o seguinte ficheiro "`alunos.csv`":

```
Número,Nome,Curso
3162,Cândido Faísca,Teatro
7777,Cristiano Ronaldo,Desporto
264,Marcelo Sousa,Ciência Política
```
Que corresponde a uma tabela com 3 registos de informação: a primeira linha de cabeçalho identifica os campos de cada registo, `Número`, `Nome`, `Curso`, e as linhas seguintes contêm os registos de informação.

No entanto, os CSV recebidos poderão conter algumas extensões cuja semântica se explica a seguir:

### 1. Listas

No cabeçalho, cada campo poderá ter um número `N` que representará o número de colunas que esse campo abrange.
Por exemplo, imaginemos que ao exemplo anterior se acrescentou um campo `Notas`, com `N = 5` ("`alunos2.csv`"):

```
Número,Nome,Curso,Notas{5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,15,16
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,18
```

Isto significa que o campo Notas abrange 5 colunas (reparem que se colocaram os campos que sobram a vazio, para o
**CSV bater certo**).

### 2. Listas com um intervalo de tamanhos

Para além de um tamanho único, podemos também definir um intervalo de tamanhos `{N, M}`, significando que o número de
colunas de um certo campo pode ir de `N` até `M` ("`alunos3.csv`"):

```
Número,Nome,Curso,Notas{3,5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
```

À semelhança do ponto anterior, havendo colunas vazias, os separadores têm de estar lá, o número de colunas deverá ser sempre igual ao valor máximo de colunas, poderão é estar preenchidas com informação ou não.

### 3. Funções de agregação

Para além de listas, podemos ter funções de agregação, aplicadas a essas listas.
Veja os seguintes exemplos: "`alunos4.csv`"

```
Número,Nome,Curso,Notas{3,5}::sum,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
```

 e "`alunos5.csv`":

 ```
Número,Nome,Curso,Notas{3,5}::media,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
 ```

## Resultados esperados

O resultado final esperado é um ficheiro JSON resultante da conversão dum ficheiro CSV.
Por exemplo, o ficheiro "`alunos.csv`" (original), deveria ser transformado no seguinte ficheiro "`alunos.json`":

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro"
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto"
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política"
  }
]
```

No caso de existirem listas, os campos que representam essas listas devem ser mapeados para listas em JSON ("`alunos2.csv ==> alunos2.json`"):

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro",
    "Notas": [12,13,14,15,16]
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto",
    "Notas": [17,12,20,11,12]
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política",
    "Notas": [18,19,19,20,18]
  }
]
```

Nos casos em que temos uma lista com uma função de agregação, o processador deve executar a função associada à lista, e
colocar o resultado no JSON, identificando na chave qual foi a função executada ("`alunos4.csv ==> alunos4.json`"):

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro",
    "Notas_sum": 39
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto",
    "Notas_sum": 72
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política",
    "Notas_sum": 76
  }
]
```

# Resposta

O trabalho foi realizado de acordo com todos os requisitos deste enunciado e implementado com todas as funcionalidades nele especificadas. As funções de agregação cuja utilização é possível são as seguintes:

-sum: Retorna o somatório de todos os elementos presentes na lista;

-prod: Retorna o produtório de todos os elementos presentes na lista;

-media: Retorna o valor médio dos valores presentes na lista;

-min: Retorna o valor mínimo presente na lista; 

-max: Retorna o valor máximo presente na lista; 

-len: Retorna o comprimento da lista;

-sorted: Retorna a lista ordenada na ordem ascendente;

-reversed: Retorna a lista na ordem reversa à original;

-unique: Retorna uma lista com todos os valores únicos presentes nela;

-summarize: Retorna um dicionário com a variância, a mediana e o desvio padrão dos valores presentes na lista;


# Código

In [1]:
import re
import json
import math

def media (lista):
    return sum(lista) / len(lista)

def unique(items):
    result = []
    for x in items:
        if x not in result:
            result.append(x)
        
    return result

def summarize(numbers):
    if not numbers:
        return {}
    mean = media(numbers)
    variance = sum((x - mean) ** 2 for x in numbers) / len(numbers)
    std_dev = variance ** 0.5
    return {"variance": variance, "median": sorted(numbers)[len(numbers)//2], "std_dev": std_dev}



def is_aggregate_function(string):
    valid_aggregate_functions = ["sum", "min", "max", "media","len", "sorted", "reversed", "summarize", "unique","prod"]
    if string in valid_aggregate_functions:
        return True
    return False

def applyFunction(funcName, value):
    if funcName == "media":
        return media(value)
    elif funcName == "summarize":
        return summarize(value)
    elif funcName == "unique":
        return unique(value)
    elif funcName == "prod":
        return math.prod(value)
    
    if funcName in  ["sum", "min", "max", "len", "sorted", "reversed"]:
        function = getattr(__builtins__, funcName)
        return function(value)
   



ficheiro = input("Insira o nome do ficheiro: ")
ficheiro = re.sub(r"\s","",ficheiro)
if not re.search(r"\.csv$",ficheiro):
    print("Erro: Programa válido apenas para ficheiros csv.")
    exit(1)

nomeSemCsv = re.sub(r"\.csv","", ficheiro)

try:
    f = open(ficheiro,"r")
except IOError: 
    print ("Erro: Ficheiro não existe.")
    exit(1)

dict = {}
dict[nomeSemCsv] = []

linha = f.readline()

if not linha:
    print("Erro: Ficheiro sem cabeçalho.")
    exit(1)
    
cabecalho = re.split(r",(?![^{]*\})",linha)

er1 = re.compile(r"^(?P<campo>\w+)({(?P<li>\d+),?(?P<ls>\d+)?})?(::(?P<funcao>[a-z]+))?$")


num_listas = 0

i = 0

cabecalho[len(cabecalho)-1] = re.sub(r"\n","",cabecalho[len(cabecalho)-1])

while i < len(cabecalho):  
    match = er1.match(cabecalho[i])
    if not match:
        print("Erro: Cabeçalho Inválido - Formato Inválido")
        exit(1)
    else:
        if match.group("li"):
            skip = int(match.group("li"))
            if skip <= 0:
                print("Erro: Cabeçalho Inválido - Lista com tamanho não positivo")
                exit(1)
            if match.group("ls"):
                ls = int(match.group("ls"))
                if skip >= ls:
                    print("Erro: Cabeçalho Inválido - Lista com limite superior menor ou igual a limite inferior")
                    exit(1)
                skip = ls
            if i + skip >= len(cabecalho):
                print("Erro: Cabeçalho Inválido - Colunas Insuficientes Depois De Uma Lista")
                exit(1)
            else:
                reservados = cabecalho[i+1:i+skip+1]
        
                for r in reservados:
                    if not re.match(r"(^$|\s)",r):
                        print("Erro: Cabeçalho Inválido - Colunas Insuficientes Depois De Uma Lista")
                        exit(1)
                i+=skip
            if match.group("funcao"):
                if not is_aggregate_function(match.group("funcao")):
                    print("Erro: Cabeçalho Inválido - Função Inválida")
                    exit(1)     
            num_listas+=1
    i+=1
 
num_linha = 2       
linha = f.readline() 

if not linha:
    print("Erro: Ficheiro sem linhas.")
    exit(1)

while linha:
    campos_linha = re.split(",",linha)
    campos_linha[len(campos_linha)-1] = re.sub(r"\n","",campos_linha[len(campos_linha)-1])
    
    if len(campos_linha) + num_listas == len(cabecalho):
        valid = True
        listas_linha = 0
        linhaDict = {}
        cab_j = 0
        li_j = 0
        while cab_j < len(cabecalho):
            matchL = er1.match(cabecalho[cab_j])
            if matchL.group("li"):
                listaNum = []
                
                lim = int(matchL.group("li"))
                
                if matchL.group("ls"):
                    lim = int(matchL.group("ls"))
                  
                if not matchL.group("funcao"):  
                    z = 0
                   
                    while z < lim:
                        if not re.match("^$",campos_linha[z+li_j]):
                            listaNum.append(float(campos_linha[z+li_j]))
                        z+=1
                    
                    if len(listaNum)<int(matchL.group("li")):
                        print("Linha " +str(num_linha)+" ignorada. Minimo de valores exigidos no campo " +matchL.group("campo")+" é " + matchL.group("li") +" e foram lidos apenas " + str(len(listaNum)) +".")
                        valid = False
                        break
                    
                    linhaDict[matchL.group("campo")] = listaNum
                    
                else:
                    funcao = matchL.group("funcao")
                    z = 0
                    while z < lim:
                        if re.match(r"(\+|-)?\d+(\.\d+)?",campos_linha[z+li_j]):
                            listaNum.append(float(campos_linha[z+li_j]))
                        z+=1
                    
                    if len(listaNum)<int(matchL.group("li")):
                        print("Linha " + str(num_linha)+" ignorada. Função de agregação '" +matchL.group("funcao")+ "' necessita de " + matchL.group("li") +" valores numéricos e tem apenas " + str(len(listaNum))+".")
                        valid = False
                        break
                    
                    result = applyFunction(funcao,listaNum)
                    linhaDict[matchL.group("campo")+"_" + matchL.group("funcao")] = result
            
                cab_j+=lim
                li_j+=lim-1
            else:
                linhaDict[cabecalho[cab_j]] = campos_linha[li_j]
                
            li_j+=1
            cab_j+=1

        if valid:
            dict[nomeSemCsv].append(linhaDict)
    else:
        print("Linha " +str(num_linha)+" ignorada. Número de valores da linha não corresponde aos exigidos pelo cabeçalho.")
        
    linha = f.readline()
    num_linha+=1
    
ficheirojson = open(nomeSemCsv+".json","w")
json.dump(dict,ficheirojson, indent=4, ensure_ascii=False)